In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
import torch

In [42]:
target_X = torch.load('data/pt-cache/target_X_split.pt')
target_y = torch.load('data/pt-cache/target_y.pt')
query_X = torch.load('data/pt-cache/query_X_split.pt')
query_y = torch.load('data/pt-cache/query_y.pt')

In [46]:
# zeros = (~torch.all(target_X.view(-1,784)==query_X.view(-1,784), dim=-1)).numpy().astype('bool')
# torch.save(torch.tensor(zeros), 'data/pt-cache/zeros.pt')
zeros = torch.load('data/pt-cache/zeros.pt')

In [49]:
target_X = target_X.view(-1,784).numpy().astype('float')
target_y = target_y.numpy().astype('int')

In [50]:
query_X = query_X.view(-1,784).numpy().astype('float')
query_y = query_y.numpy().astype('int')

In [51]:
sample_X = (np.load('data/sample_images.npy')/127.5-1)[:9,:].reshape(-1,784)
sample_y = np.arange(sample_X.shape[0])

In [11]:
# pca_target = PCA(n_components=0.99)
# target_pca = pca_target.fit_transform(target_X)
# torch.save(pca_target, 'data/pt-cache/pca_target.pt')
pca_target = torch.load('data/pt-cache/pca_target.pt')
target_pca = pca_target.transform(target_X)
sample_pca = pca_target.transform(sample_X)

In [52]:
# kmeans_target = KMeans(n_clusters=8)
# target_pred = kmeans_target.fit_predict(target_pca)
# torch.save(kmeans_target, 'data/pt-cache/kmeans_target.pt')
kmeans_target = torch.load('data/pt-cache/kmeans_target.pt')
sample_pred = kmeans_target.predict(sample_pca)
cluster_map = np.empty(8).astype('int')
cluster_map[sample_pred] = sample_y
target_pred = cluster_map[kmeans_target.predict(target_pca)]
torch.save(torch.tensor(target_pred), 'data/pt-cache/target_pred.pt')
print(classification_report(target_y, target_pred))

              precision    recall  f1-score   support

           1       0.77      0.93      0.84     80000
           2       0.92      0.87      0.89     80000
           3       0.99      0.87      0.93     80000
           4       0.54      0.56      0.55     80000
           5       0.96      0.80      0.87     80000
           6       0.93      0.88      0.90     80000
           7       0.51      0.56      0.54     80000
           8       0.90      0.94      0.92     80000

    accuracy                           0.80    640000
   macro avg       0.81      0.80      0.81    640000
weighted avg       0.81      0.80      0.81    640000



In [53]:
query_pca = pca_target.transform(query_X)
query_pred_adv = cluster_map[kmeans_target.predict(query_pca)]
query_pred_adv[zeros] = 0
torch.save(torch.tensor(query_pred_adv), 'data/pt-cache/query_pred_adv.pt')
print(classification_report(query_y, query_pred_adv))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    316413
           1       0.76      0.93      0.84     40470
           2       0.92      0.87      0.89     40442
           3       0.99      0.88      0.93     40303
           4       0.54      0.56      0.55     40542
           5       0.96      0.80      0.87     40424
           6       0.93      0.88      0.90     40245
           7       0.51      0.56      0.54     40765
           8       0.90      0.94      0.92     40396

    accuracy                           0.90    640000
   macro avg       0.83      0.82      0.83    640000
weighted avg       0.91      0.90      0.90    640000



In [54]:
# pca_query = PCA(n_components=0.99)
# query_pca = pca_query.fit_transform(query_X)
# torch.save(pca_query, 'data/pt-cache/pca_query.pt')
pca_query = torch.load('data/pt-cache/pca_query.pt')
query_pca = pca_query.transform(query_X)
sample_pca = pca_query.transform(sample_X)

In [55]:
kmeans_query = KMeans(n_clusters=9)
query_pred = kmeans_query.fit_predict(query_pca)
torch.save(kmeans_query, 'data/pt-cache/kmeans_query.pt')
kmeans_query = torch.load('data/pt-cache/kmeans_query.pt')
sample_pred = kmeans_query.predict(sample_pca)
cluster_map = np.empty(9).astype('int')
cluster_map[sample_pred] = sample_y
query_pred = cluster_map[kmeans_query.predict(query_pca)]
torch.save(torch.tensor(query_pred), 'data/pt-cache/query_pred.pt')
print(classification_report(query_y, query_pred))

/home/apps/anaconda3_2018/4.6.9/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apps/anaconda3_2018/4.6.9/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.20      0.34    316413
           1       0.00      0.00      0.00     40470
           2       0.00      0.00      0.00     40442
           3       0.90      0.86      0.88     40303
           4       0.00      0.00      0.00     40542
           5       0.52      0.96      0.68     40424
           6       0.86      0.92      0.89     40245
           7       0.45      0.90      0.60     40765
           8       0.36      0.99      0.53     40396
         186       0.00      0.00      0.00         0
         207       0.00      0.00      0.00         0
         293       0.00      0.00      0.00         0

    accuracy                           0.39    640000
   macro avg       0.34      0.40      0.33    640000
weighted avg       0.69      0.39      0.39    640000

